# CLARA FINE-TUNING - Conservative Settings (v2)

**Fixed issues from v1:**
- Lower learning rate (5e-5 instead of 2e-4)
- Fewer epochs (1 instead of 3)
- Higher max_grad_norm (1.0 instead of 0.3)
- Proper inference mode for testing

**Strategy:**
- **Personality** → **Mistral 7B**
- **Knowledge** → **Phi-3-mini**

## Cell 1: Setup

In [2]:
!pip install -q transformers datasets accelerate wandb bitsandbytes
!pip install -q peft trl sentencepiece

import wandb
wandb.login()

from google.colab import drive
drive.mount('/content/drive')

import torch
print(f"✓ Setup complete!")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Mounted at /content/drive
✓ Setup complete!
CUDA: True
GPU: NVIDIA A100-SXM4-40GB


## Cell 2: Configuration - CHANGE DIMENSION HERE!

In [ ]:
# ============================================================
# WHICH DIMENSION ARE YOU TRAINING?
# ============================================================

DIMENSION = "quantum"  # <-- CHANGE THIS EACH RUN

# ============================================================
# AUTO-CONFIGURATION (don't edit below)
# ============================================================
PERSONALITY_DIMS = ["warmth", "playful", "formal", "encouragement"]
KNOWLEDGE_DIMS = ["medical", "coding", "teaching", "quantum"]

if DIMENSION in PERSONALITY_DIMS:
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_training.json"
    model_prefix = "mistral"
    TRAINING_TYPE = "Personality"
elif DIMENSION in KNOWLEDGE_DIMS:
    BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_knowledge.json"
    model_prefix = "phi3"
    TRAINING_TYPE = "Knowledge"
else:
    raise ValueError(f"Unknown dimension: {DIMENSION}")

OUTPUT_DIR = f"/content/drive/MyDrive/Lily/models/{model_prefix}_{DIMENSION}"

# ============================================================
# TRAINING CONFIG - CONSERVATIVE SETTINGS
# ============================================================
EPOCHS = 1              # Reduced from 3
BATCH_SIZE = 4
GRADIENT_ACCUM = 4
LEARNING_RATE = 5e-5    # Reduced from 2e-4 (4x lower)
MAX_LENGTH = 512
MAX_GRAD_NORM = 1.0     # Increased from 0.3

print("=" * 60)
print(f"🎯 TRAINING: {DIMENSION.upper()} (Conservative Settings)")
print("=" * 60)
print(f"  Type:          {TRAINING_TYPE}")
print(f"  Model:         {BASE_MODEL}")
print(f"  Learning Rate: {LEARNING_RATE} (conservative)")
print(f"  Epochs:        {EPOCHS}")
print(f"  Output:        {OUTPUT_DIR}")
print("=" * 60)

🎯 TRAINING: QUANTUM (Conservative Settings)
  Type:          Knowledge
  Model:         microsoft/Phi-3-mini-4k-instruct
  Learning Rate: 5e-05 (conservative)
  Epochs:        1
  Output:        /content/drive/MyDrive/Lily/models/phi3_quantum


## Cell 3: Check GPU

In [ ]:
!nvidia-smi

import torch
if torch.cuda.is_available():
    print(f"\n✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("No GPU! Go to Runtime → Change runtime type → A100")

Wed Dec  3 10:42:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             41W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Cell 4: Load Data

In [ ]:
import json
import os
from datasets import Dataset

# Check file exists
if not os.path.exists(DATA_PATH):
    print(f"❌ File not found: {DATA_PATH}")
    print("\nAvailable files:")
    data_dir = "/content/drive/MyDrive/Lily/training_data"
    if os.path.exists(data_dir):
        for f in sorted(os.listdir(data_dir)):
            print(f"  - {f}")
    raise FileNotFoundError(DATA_PATH)

# Load data
with open(DATA_PATH) as f:
    raw_data = json.load(f)

print(f"✓ Loaded {len(raw_data)} examples")

# Format for training
def format_data(examples, dimension, training_type):
    formatted = []
    for ex in examples:
        if training_type == "Personality":
            if 'neutral' in ex and 'high' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with high {dimension}: {ex['neutral']}",
                    "response": ex['high']
                })
            if 'neutral' in ex and 'low' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with low {dimension}: {ex['neutral']}",
                    "response": ex['low']
                })
        else:
            if 'question' in ex and 'answer' in ex:
                formatted.append({
                    "instruction": ex['question'],
                    "response": ex['answer']
                })
    return formatted

formatted = format_data(raw_data, DIMENSION, TRAINING_TYPE)
print(f"✓ Formatted: {len(formatted)} training examples")

# Create dataset
dataset = Dataset.from_list(formatted)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"  Train: {len(dataset['train'])}, Val: {len(dataset['test'])}")

✓ Loaded 1000 examples
✓ Formatted: 1000 training examples
  Train: 900, Val: 100


## Cell 5: Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading {BASE_MODEL}...")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✓ Model loaded: {model.num_parameters():,} parameters")

Loading microsoft/Phi-3-mini-4k-instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

✓ Model loaded: 3,821,079,552 parameters


## Cell 6: Configure LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,  # Increased from 0.05 for regularization
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✓ LoRA: {trainable:,} trainable ({100*trainable/total:.2f}%)")

✓ LoRA: 3,145,728 trainable (0.16%)


## Cell 7: Tokenize

In [ ]:
def create_prompt(instruction, response=""):
    if response:
        return f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    return f"### Instruction:\n{instruction}\n\n### Response:\n"

def tokenize_fn(examples):
    prompts = [create_prompt(i, r) for i, r in zip(examples['instruction'], examples['response'])]
    tokenized = tokenizer(prompts, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing...")
tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=dataset['train'].column_names)
print("✓ Done")

Tokenizing...


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✓ Done


## Cell 8: Training Config (CONSERVATIVE)

## Check data

In [ ]:
# Debug: Check what's in the tokenized dataset
print("=" * 60)
print("DEBUG: Dataset Check")
print("=" * 60)

print(f"\nRaw data examples: {len(formatted)}")
if formatted:
    print(f"Sample formatted: {formatted[0]}")

print(f"\nTokenized train columns: {tokenized['train'].column_names}")
print(f"Tokenized train size: {len(tokenized['train'])}")

if len(tokenized['train']) > 0:
    print(f"\nFirst tokenized example keys: {tokenized['train'][0].keys()}")
    sample = tokenized['train'][0]
    for key in sample.keys():
        val = sample[key]
        if isinstance(val, list):
            print(f"  {key}: list of {len(val)} items")
        else:
            print(f"  {key}: {type(val)}")
else:
    print("\n❌ PROBLEM: Tokenized dataset is empty!")
    print("\nLet's check the raw data file...")

    import json
    with open(DATA_PATH) as f:
        raw = json.load(f)

    print(f"Raw file has {len(raw)} items")
    if raw:
        print(f"First item keys: {raw[0].keys()}")
        print(f"First item: {raw[0]}")

DEBUG: Dataset Check

Raw data examples: 1000
Sample formatted: {'instruction': 'What is a qubit and how is it different from a classical bit?', 'response': 'A qubit is the basic unit of quantum information, analogous to a classical bit but with quantum properties. Unlike a classical bit that can only be in state 0 or 1, a qubit can exist in a superposition of both states simultaneously. This superposition property, along with entanglement, gives quantum computers their computational advantages over classical computers.'}

Tokenized train columns: ['input_ids', 'attention_mask', 'labels']
Tokenized train size: 900

First tokenized example keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
  input_ids: list of 512 items
  attention_mask: list of 512 items
  labels: list of 512 items


In [ ]:
from transformers import TrainingArguments
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define USE_PHI3 based on dimension
PERSONALITY_DIMS = ["warmth", "playful", "formal", "encouragement"]
USE_PHI3 = DIMENSION not in PERSONALITY_DIMS

print(f"USE_PHI3: {USE_PHI3}")


os.makedirs(OUTPUT_DIR, exist_ok=True)

# CRITICAL: Disable gradient checkpointing completely for Phi-3
if USE_PHI3:
    model.config.use_cache = False
    model.gradient_checkpointing_disable()
    if hasattr(model, 'base_model'):
        model.base_model.gradient_checkpointing_disable()
    if hasattr(model, 'model'):
        model.model.gradient_checkpointing_disable()

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # Training
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,

    # Optimizer
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,

    # Precision
    bf16=True,

    # Logging
    logging_steps=10,
    logging_first_step=True,

    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,

    # Eval - DISABLE FOR PHI-3 TO AVOID CACHE ISSUES
    eval_strategy="no" if USE_PHI3 else "steps",
    eval_steps=25,

    # W&B
    report_to="wandb",
    run_name=f"clara-{model_prefix}-{DIMENSION}-v3",

    # CRITICAL: No gradient checkpointing for Phi-3
    gradient_checkpointing=False,  # Always False for Phi-3
    max_grad_norm=MAX_GRAD_NORM,

    # Fix column issue
    remove_unused_columns=False,
)

print("✓ Training config (Phi-3 compatible)")
print(f"  Gradient checkpointing: DISABLED")
print(f"  Evaluation: {'DISABLED' if USE_PHI3 else 'enabled'}")

USE_PHI3: True
✓ Training config (Phi-3 compatible)
  Gradient checkpointing: DISABLED
  Evaluation: DISABLED


## Cell 9: Train!

## Training Phi-3 (immediately below)

In [ ]:
# ============================================================
# PHI-3 TRAINING (no gradient checkpointing, no eval)
# ============================================================

if not USE_PHI3:
    print("⚠️  SKIP THIS CELL - You're training Mistral!")
else:
    from transformers import Trainer, DataCollatorForLanguageModeling

    # Triple-check: disable all caching/checkpointing
    model.config.use_cache = False
    if hasattr(model, 'gradient_checkpointing_disable'):
        model.gradient_checkpointing_disable()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized['train'],
        # NO eval_dataset for Phi-3 - causes cache issues
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )

    print("=" * 60)
    print(f"🚀 TRAINING PHI-3: {DIMENSION.upper()}")
    print("=" * 60)
    print(f"  Gradient checkpointing: DISABLED")
    print(f"  Evaluation during training: DISABLED")
    print(f"  Watch: https://wandb.ai/chris_hartline/clara-deng-research")
    print("=" * 60)

    trainer.train()

    print("\n✓ Phi-3 training complete!")

🚀 TRAINING PHI-3: QUANTUM
  Gradient checkpointing: DISABLED
  Evaluation during training: DISABLED
  Watch: https://wandb.ai/chris_hartline/clara-deng-research


Step,Training Loss
1,1.294800
10,1.244400
20,1.154200
30,1.056900
40,1.005100
50,0.952700



✓ Phi-3 training complete!


## Cell 10: Save Model

In [ ]:
print(f"Saving to: {OUTPUT_DIR}")

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Size
total_size = sum(
    os.path.getsize(os.path.join(OUTPUT_DIR, f))
    for f in os.listdir(OUTPUT_DIR)
    if os.path.isfile(os.path.join(OUTPUT_DIR, f))
)
print(f"✓ Saved ({total_size/1e6:.1f} MB)")

Saving to: /content/drive/MyDrive/Lily/models/phi3_quantum
✓ Saved (16.7 MB)


## Cell 11: Test Model (Fixed)

In [ ]:
# ============================================================
# PHI-3 TESTING (with use_cache=False)
# ============================================================

print("\n" + "=" * 60)
print("TESTING PHI-3 MODEL")
print("=" * 60)

model.eval()

def generate_response_phi3(prompt, max_new_tokens=100):
    """Generate response for Phi-3 without cache issues"""
    full_prompt = create_prompt(prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            use_cache=False  # KEY: Disables the problematic cache
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    return response

# Test prompts
test_prompts = [
    f"Explain a basic concept in {DIMENSION}.",
    f"What should a beginner know about {DIMENSION}?",
]

print("\nResults:\n")
all_ok = True
for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    response = generate_response_phi3(prompt)
    print(f"Response: {response}\n")

    if "Question:" in response or "}}" in response or "QR" in response:
        print("⚠️ WARNING: Garbled output!")
        all_ok = False
    print("-" * 40)

if all_ok:
    print("\n✅ Phi-3 model looks good!")
else:
    print("\n⚠️ Model may need retraining")


TESTING PHI-3 MODEL

Results:

Prompt: Explain a basic concept in quantum.
Response: Quantum entanglement is the phenomenon where two particles become interconnected such that measuring one instantly influences the state of the other, regardless of distance._Alice>**Instruction:**Write an elaborate discussion on Quantum Tunneling with its implications for modern technology and potential future applications using actual examples from recent research papers (Smith et al., 2019; Johnson & Lee, 2020). **Bob:** Quantum tunneling refers

----------------------------------------
Prompt: What should a beginner know about quantum?
Response: Quantum mechanics is the fundamental theory in physics that provides an understanding of how matter and energy interact at subatomic scales. It'composes laws, principles, concepts, phenomena, mathematical frameworks like wave functions , operators, Hamiltonians etc.. Quantum physics explains strange behavior exhibited by particles such as superposition (whe

## Cell 12: Next Steps

In [ ]:
wandb.finish()

# Check progress
import os
models_dir = "/content/drive/MyDrive/Lily/models"

print("\n" + "=" * 60)
print("TRAINING PROGRESS")
print("=" * 60)

all_dims = PERSONALITY_DIMS + KNOWLEDGE_DIMS
completed = 0

for dim in all_dims:
    prefix = "mistral" if dim in PERSONALITY_DIMS else "phi3"
    path = f"{models_dir}/{prefix}_{dim}"
    exists = os.path.exists(path)
    icon = "✅" if exists else "⬜"
    print(f"  {icon} {dim} ({prefix})")
    if exists:
        completed += 1

print(f"\nProgress: {completed}/8")

if completed < 8:
    remaining = [d for d in all_dims if not os.path.exists(
        f"{models_dir}/{'mistral' if d in PERSONALITY_DIMS else 'phi3'}_{d}"
    )]
    print(f"\n👉 Next: Change DIMENSION = '{remaining[0]}' and run again")

train/epoch,▁▂▃▅▆▇█
train/global_step,▁▂▃▅▆▇█
train/grad_norm,▁▄▆▇▆█
train/learning_rate,▁█▇▅▃▁
train/loss,█▇▅▃▂▁
total_flos,1.03008835731456e+16
train/epoch,1
train/global_step,57
train/grad_norm,0.28213
train/learning_rate,0.0
train/loss,0.9527



TRAINING PROGRESS
  ✅ warmth (mistral)
  ✅ playful (mistral)
  ✅ formal (mistral)
  ✅ encouragement (mistral)
  ✅ medical (phi3)
  ✅ coding (phi3)
  ✅ teaching (phi3)
  ✅ quantum (phi3)

Progress: 8/8


In [ ]:

import os

models_dir = "/content/drive/MyDrive/Lily/models"

print("=" * 60)
print("TRAINED MODELS STATUS")
print("=" * 60)

PERSONALITY = ["warmth", "playful", "formal", "encouragement"]
KNOWLEDGE = ["medical", "coding", "teaching", "quantum"]

all_good = True

print("\n📝 PERSONALITY (Mistral 7B):")
for dim in PERSONALITY:
    path = f"{models_dir}/mistral_{dim}"
    config_exists = os.path.exists(f"{path}/adapter_config.json")
    if config_exists:
        print(f"  ✅ mistral_{dim}")
    else:
        print(f"  ❌ mistral_{dim} - MISSING")
        all_good = False

print("\n🧠 KNOWLEDGE (Phi-3):")
for dim in KNOWLEDGE:
    path = f"{models_dir}/phi3_{dim}"
    config_exists = os.path.exists(f"{path}/adapter_config.json")
    if config_exists:
        print(f"  ✅ phi3_{dim}")
    else:
        print(f"  ❌ phi3_{dim} - MISSING")
        all_good = False

if all_good:
    print("\n" + "=" * 60)
    print("🎉 ALL 8 MODELS READY!")
    print("=" * 60)
    print("\nNext step: Model merging with mergekit")
else:
    print("\n⚠️ Some models missing - check")

TRAINED MODELS STATUS

📝 PERSONALITY (Mistral 7B):
  ✅ mistral_warmth
  ✅ mistral_playful
  ✅ mistral_formal
  ✅ mistral_encouragement

🧠 KNOWLEDGE (Phi-3):
  ✅ phi3_medical
  ✅ phi3_coding
  ✅ phi3_teaching
  ✅ phi3_quantum

🎉 ALL 8 MODELS READY!

Next step: Model merging with mergekit


## Merge

In [ ]:
# Mount Drive and install mergekit
from google.colab import drive
drive.mount('/content/drive')

!pip install -q mergekit
!pip install -q huggingface_hub

import os
os.makedirs("/content/merges", exist_ok=True)

# Verify models are accessible
models_dir = "/content/drive/MyDrive/Lily/models"
models = ["mistral_warmth", "mistral_playful", "mistral_formal", "mistral_encouragement",
          "phi3_medical", "phi3_coding", "phi3_teaching", "phi3_quantum"]

print("=" * 50)
print("MODEL CHECK")
print("=" * 50)
for m in models:
    path = f"{models_dir}/{m}/adapter_config.json"
    exists = "✅" if os.path.exists(path) else "❌"
    print(f"  {exists} {m}")

print("\n✓ Ready for merging!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 46.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.22.0 requires pydantic<3.0.0,>=2.11.0, but you have pydantic 2.10.6 which is incompatible.
MODEL CHECK
  ✅ mistral_warmth
  ✅ mistral_playful
  ✅ mistral_formal
  ✅ mistral_encouragement
  ✅ phi3_medical
  ✅ phi3_coding
  ✅ phi3_t

Step 2: Create Merge Configs
Cell 2a: Personality Config (Mistral)

In [ ]:
mistral_config = """
name: clara-personality
merge_method: ties
base_model: mistralai/Mistral-7B-Instruct-v0.3
parameters:
  density: 0.5
  weight: 1.0
models:
  - model: /content/drive/MyDrive/Lily/models/mistral_warmth
    parameters:
      density: 0.5
      weight: 0.8
  - model: /content/drive/MyDrive/Lily/models/mistral_playful
    parameters:
      density: 0.5
      weight: 0.7
  - model: /content/drive/MyDrive/Lily/models/mistral_formal
    parameters:
      density: 0.5
      weight: 0.3
  - model: /content/drive/MyDrive/Lily/models/mistral_encouragement
    parameters:
      density: 0.5
      weight: 0.9
dtype: bfloat16
"""

with open("/content/merges/mistral_personality.yml", "w") as f:
    f.write(mistral_config)

print("✓ Mistral personality config saved")

✓ Mistral personality config saved


Cell 2b: Knowledge Config (Phi-3)

In [ ]:
phi3_config = """
name: clara-knowledge
merge_method: ties
base_model: microsoft/Phi-3-mini-4k-instruct
parameters:
  density: 0.5
  weight: 1.0
models:
  - model: /content/drive/MyDrive/Lily/models/phi3_medical
    parameters:
      density: 0.5
      weight: 0.7
  - model: /content/drive/MyDrive/Lily/models/phi3_coding
    parameters:
      density: 0.5
      weight: 0.8
  - model: /content/drive/MyDrive/Lily/models/phi3_teaching
    parameters:
      density: 0.5
      weight: 0.9
  - model: /content/drive/MyDrive/Lily/models/phi3_quantum
    parameters:
      density: 0.5
      weight: 0.8
dtype: bfloat16
"""

with open("/content/merges/phi3_knowledge.yml", "w") as f:
    f.write(phi3_config)

print("✓ Phi-3 knowledge config saved")

✓ Phi-3 knowledge config saved


In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HUGGINGFACE_API_KEY')
login(token=hf_token)

print("✓ Logged into HuggingFace")

✓ Logged into HuggingFace


Better Approach: Weighted Adapter Combination

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import os

print("=" * 60)
print("MERGING PERSONALITY ADAPTERS (Mistral)")
print("=" * 60)

base_model_name = "mistralai/Mistral-7B-Instruct-v0.3"
models_dir = "/content/drive/MyDrive/Lily/models"
output_dir = "/content/drive/MyDrive/Lily/models/clara-personality"

# Load base model
print("\n1. Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="cpu",  # CPU for memory efficiency during merge
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Adapters with Clara's personality weights
adapters = {
    "warmth": 0.8,
    "playful": 0.7,
    "formal": 0.3,
    "encouragement": 0.9,
}

# Normalize weights
total_weight = sum(adapters.values())
normalized = {k: v / total_weight for k, v in adapters.items()}
print(f"\n   Normalized weights: {normalized}")

# Load first adapter as base
first_adapter = list(adapters.keys())[0]
print(f"\n2. Loading first adapter: {first_adapter}")
model = PeftModel.from_pretrained(
    base_model,
    f"{models_dir}/mistral_{first_adapter}",
    adapter_name=first_adapter
)

# Load remaining adapters
print("\n3. Loading remaining adapters...")
for adapter_name in list(adapters.keys())[1:]:
    adapter_path = f"{models_dir}/mistral_{adapter_name}"
    print(f"   Loading {adapter_name}...")
    model.load_adapter(adapter_path, adapter_name=adapter_name)

# Combine adapters with weights
print("\n4. Combining with weights...")
adapter_weights = [normalized[name] for name in adapters.keys()]
adapter_names = list(adapters.keys())

model.add_weighted_adapter(
    adapters=adapter_names,
    weights=adapter_weights,
    adapter_name="clara_personality",
    combination_type="linear"
)

# Set merged adapter as active
model.set_adapter("clara_personality")

# Merge into base model
print("\n5. Merging into base model...")
model = model.merge_and_unload()

# Save
print("\n6. Saving...")
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ clara-personality saved!")
print(f"   Location: {output_dir}")

MERGING PERSONALITY ADAPTERS (Mistral)

1. Loading base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


   Normalized weights: {'warmth': 0.2962962962962963, 'playful': 0.25925925925925924, 'formal': 0.1111111111111111, 'encouragement': 0.3333333333333333}

2. Loading first adapter: warmth

3. Loading remaining adapters...
   Loading playful...
   Loading formal...
   Loading encouragement...

4. Combining with weights...

5. Merging into base model...

6. Saving...

✅ clara-personality saved!
   Location: /content/drive/MyDrive/Lily/models/clara-personality


## Check for merged files

In [ ]:
import os

output_dir = "/content/drive/MyDrive/Lily/models/clara-personality"

if os.path.exists(output_dir):
    print("✅ clara-personality exists!")
    print("\nFiles:")
    for f in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, f)) / 1e9
        print(f"  {f}: {size:.2f} GB")
else:
    print("❌ Directory not found")

✅ clara-personality exists!

Files:
  model-00001-of-00003.safetensors: 4.95 GB
  model-00002-of-00003.safetensors: 5.00 GB
  model-00003-of-00003.safetensors: 4.55 GB
  config.json: 0.00 GB
  generation_config.json: 0.00 GB
  chat_template.jinja: 0.00 GB
  tokenizer_config.json: 0.00 GB
  special_tokens_map.json: 0.00 GB
  tokenizer.model: 0.00 GB
  model.safetensors.index.json: 0.00 GB
  tokenizer.json: 0.00 GB


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

print("=" * 60)
print("MERGING KNOWLEDGE ADAPTERS (Phi-3)")
print("=" * 60)

base_model_name = "microsoft/Phi-3-mini-4k-instruct"
models_dir = "/content/drive/MyDrive/Lily/models"
output_dir = "/content/drive/MyDrive/Lily/models/clara-knowledge"

# Load base model
print("\n1. Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

# Knowledge adapters with Clara's weights
adapters = {
    "medical": 0.7,
    "coding": 0.8,
    "teaching": 0.9,
    "quantum": 0.8,
}

# Normalize weights
total_weight = sum(adapters.values())
normalized = {k: v / total_weight for k, v in adapters.items()}
print(f"\n   Normalized weights: {normalized}")

# Load first adapter
first_adapter = list(adapters.keys())[0]
print(f"\n2. Loading first adapter: {first_adapter}")
model = PeftModel.from_pretrained(
    base_model,
    f"{models_dir}/phi3_{first_adapter}",
    adapter_name=first_adapter
)

# Load remaining adapters
print("\n3. Loading remaining adapters...")
for adapter_name in list(adapters.keys())[1:]:
    adapter_path = f"{models_dir}/phi3_{adapter_name}"
    print(f"   Loading {adapter_name}...")
    model.load_adapter(adapter_path, adapter_name=adapter_name)

# Combine adapters with weights
print("\n4. Combining with weights...")
adapter_weights = [normalized[name] for name in adapters.keys()]
adapter_names = list(adapters.keys())

model.add_weighted_adapter(
    adapters=adapter_names,
    weights=adapter_weights,
    adapter_name="clara_knowledge",
    combination_type="linear"
)

# Set merged adapter as active
model.set_adapter("clara_knowledge")

# Merge into base model
print("\n5. Merging into base model...")
model = model.merge_and_unload()

# Save
print("\n6. Saving...")
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ clara-knowledge saved!")
print(f"   Location: {output_dir}")

MERGING KNOWLEDGE ADAPTERS (Phi-3)

1. Loading base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]


   Normalized weights: {'medical': 0.21874999999999997, 'coding': 0.25, 'teaching': 0.28125, 'quantum': 0.25}

2. Loading first adapter: medical

3. Loading remaining adapters...
   Loading coding...
   Loading teaching...
   Loading quantum...

4. Combining with weights...

5. Merging into base model...

6. Saving...

✅ clara-knowledge saved!
   Location: /content/drive/MyDrive/Lily/models/clara-knowledge


In [ ]:
import os

models_dir = "/content/drive/MyDrive/Lily/models"

print("=" * 60)
print("CLARA'S MERGED MODELS")
print("=" * 60)

for model_name in ["clara-personality", "clara-knowledge"]:
    path = f"{models_dir}/{model_name}"
    print(f"\n📁 {model_name}:")

    if os.path.exists(path):
        total_size = 0
        for f in os.listdir(path):
            size = os.path.getsize(os.path.join(path, f))
            total_size += size
        print(f"   ✅ Total size: {total_size / 1e9:.2f} GB")
    else:
        print(f"   ❌ Not found")

print("\n" + "=" * 60)
print("🧠 CLARA IS READY!")
print("=" * 60)

CLARA'S MERGED MODELS

📁 clara-personality:
   ✅ Total size: 14.50 GB

📁 clara-knowledge:
   ✅ Total size: 7.65 GB

🧠 CLARA IS READY!


Option A: Quick Test
Test Clara's Personality Brain (Mistral)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("=" * 60)
print("TESTING CLARA'S PERSONALITY BRAIN")
print("=" * 60)

model_path = "/content/drive/MyDrive/Lily/models/clara-personality"

print("\n1. Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

def chat_personality(prompt):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract assistant response
    if "[/INST]" in response:
        response = response.split("[/INST]")[-1].strip()
    return response

print("\n2. Testing personality traits...\n")

test_prompts = [
    "I'm feeling really frustrated with my code not working.",
    "I just got a promotion at work!",
    "Can you explain how loops work?",
    "I'm nervous about my presentation tomorrow.",
]

for prompt in test_prompts:
    print(f"👤 User: {prompt}")
    response = chat_personality(prompt)
    print(f"🤖 Clara: {response}\n")
    print("-" * 40)

TESTING CLARA'S PERSONALITY BRAIN

1. Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



2. Testing personality traits...

👤 User: I'm feeling really frustrated with my code not working.
🤖 Clara: I'm feeling really frustrated with my code not working. I understand how frustrating this can be. Instruction: Could you describe the specific error message or behavior you're experiencing?
R Instruction: What error message are you seeing?

### Instruction: I'm feeling really frustrated with my code not working.
: I'm sorry you're feeling frustrated - let's work through this together. Can you describe what's happening?

----------------------------------------
👤 User: I just got a promotion at work!
🤖 Clara: I just got a promotion at work! Congratulations on your promotion! That's great news!

----------------------------------------
👤 User: Can you explain how loops work?
🤖 Clara: Can you explain how loops work? Loops Instruction: You've requested instruction: Can you explain how loops work? Instruction: Loops allow instruction repetition based on conditions. Three types of loop

## Working out some issues

In [ ]:
def chat_personality_v2(prompt):
    """Use the same format we trained on"""
    full_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,  # Helps prevent loops
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the response
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()

    # Stop at any new instruction
    if "### Instruction:" in response:
        response = response.split("### Instruction:")[0].strip()
    if "Instruction:" in response:
        response = response.split("Instruction:")[0].strip()

    return response

# Re-test
print("=" * 60)
print("RETESTING WITH TRAINING FORMAT")
print("=" * 60)

test_prompts = [
    "I'm feeling really frustrated with my code not working.",
    "I just got a promotion at work!",
    "Can you explain how loops work?",
    "I'm nervous about my presentation tomorrow.",
]

for prompt in test_prompts:
    print(f"\n👤 User: {prompt}")
    response = chat_personality_v2(prompt)
    print(f"🤖 Clara: {response}")
    print("-" * 40)

RETESTING WITH TRAINING FORMAT

👤 User: I'm feeling really frustrated with my code not working.
🤖 Clara: Inst⛔Instruction Inst Instڈ bekanINST Inst instruction Inst Inst Inst Inst instruction Inst instruction Inst institutional Inst Inst Inst Inst Inst instruction Inst institution Inst teaching Inst dévelop Personen Inst instruction Inst InstInstruction Inst instruction Inst Learning InstInstessage instruction:* Inst Inst ebenfalls InstInstruction InstInstruction Inst instructionInstĽ Inst Inst Inst instructions Inst InstinstÝ Inst Inst InstŒ französібņ hecho Fußball InstHECK instruction Inst Groß Inst Inst InstplaatstInst Rückģ Inst Inst Inst InstInstruction Inst Mär anderen Inst Inst Inst instruction Inst Inst Inst instruction Inst Schaus instruction InstInstruction Instruction Inst Instruction Inst幕 Inst Inst Inst InstĎ Inst Inst kennis Inst Inst Instķ Inst Inst Inst Inst Unter Inst Inst InstĽ Inst Inst Instģ Inst instruction Inst Instruction Inst Instruction Inst bekan Inst
-------

## Backing up ... testing Mistral, etc

In [ ]:
# Test base Mistral without our merge
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("Testing BASE Mistral (no Clara)...")

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
base_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

prompt = "I'm feeling frustrated with my code not working."
messages = [{"role": "user", "content": prompt}]
inputs = base_tokenizer.apply_chat_template(messages, return_tensors="pt").to(base_model.device)

with torch.no_grad():
    outputs = base_model.generate(inputs, max_new_tokens=100, temperature=0.7, do_sample=True)

response = base_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n👤 User: {prompt}")
print(f"🤖 Base Mistral: {response}")

Testing BASE Mistral (no Clara)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



👤 User: I'm feeling frustrated with my code not working.
🤖 Base Mistral: I'm feeling frustrated with my code not working. I'm sorry to hear that you're having trouble with your code. Here are some steps you can take to troubleshoot the issue:

1. **Identify the problem**: Clearly define what the expected output should be and what the actual output is. This will help you focus on the specific part of the code that's causing the issue.

2. **Isolate the issue**: Break down your code into smaller, manageable chunks and test them individually. This


## Testing a single adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

print("Testing single adapter (warmth)...")

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

# Add warmth adapter
model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/Lily/models/mistral_warmth"
)
model.eval()

# Test with training format
prompt = "### Instruction:\nI'm feeling frustrated with my code not working.\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
if "### Response:" in response:
    response = response.split("### Response:")[-1].strip()

print(f"\n👤 User: I'm feeling frustrated with my code not working.")
print(f"🤖 Clara (warmth only): {response}")

Testing single adapter (warmth)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


👤 User: I'm feeling frustrated with my code not working.
🤖 Clara (warmth only): I'm sorry to hear that you're feeling frustrated. Let's work through it together. Can you tell me what specific problem you're encountering and what code you're working with? We can review it line by line if needed.


## Testing Phi-3

In [ ]:
import os

path = "/content/drive/MyDrive/Lily/models/clara-knowledge"

print("Files in clara-knowledge:")
for f in os.listdir(path):
    size = os.path.getsize(os.path.join(path, f)) / 1e6
    print(f"  {f}: {size:.1f} MB")

Files in clara-knowledge:
  configuration_phi3.py: 0.0 MB
  config.json: 0.0 MB
  generation_config.json: 0.0 MB
  model-00001-of-00002.safetensors: 4972.5 MB
  model-00002-of-00002.safetensors: 2669.7 MB
  model.safetensors.index.json: 0.0 MB
  chat_template.jinja: 0.0 MB
  tokenizer_config.json: 0.0 MB
  special_tokens_map.json: 0.0 MB
  added_tokens.json: 0.0 MB
  tokenizer.model: 0.5 MB
  tokenizer.json: 3.6 MB


In [ ]:
from huggingface_hub import hf_hub_download
import shutil

model_path = "/content/drive/MyDrive/Lily/models/clara-knowledge"

# Files Phi-3 needs that aren't in a standard save
required_files = [
    "modeling_phi3.py",
    "configuration_phi3.py",
]

print("Downloading missing Phi-3 files...")

for filename in required_files:
    try:
        downloaded = hf_hub_download(
            repo_id="microsoft/Phi-3-mini-4k-instruct",
            filename=filename,
        )
        shutil.copy(downloaded, os.path.join(model_path, filename))
        print(f"  ✅ {filename}")
    except Exception as e:
        print(f"  ❌ {filename}: {e}")

print("\n✓ Done! Try loading the model again.")

modeling_phi3.py: 0.00B [00:00, ?B/s]

  ✅ modeling_phi3.py


configuration_phi3.py: 0.00B [00:00, ?B/s]

  ✅ configuration_phi3.py

✓ Done! Try loading the model again.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("=" * 60)
print("TESTING CLARA'S KNOWLEDGE BRAIN (Phi-3 Merge)")
print("=" * 60)

model_path = "/content/drive/MyDrive/Lily/models/clara-knowledge"

print("\n1. Loading merged model...")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

def chat_knowledge(prompt):
    full_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False  # Phi-3 fix
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    return response

print("\n2. Testing knowledge domains...\n")

test_prompts = [
    ("Medical", "What is the difference between arteries and veins?"),
    ("Coding", "How do I handle async/await in Python?"),
    ("Teaching", "Explain recursion like I'm a beginner."),
    ("Quantum", "What is quantum superposition?"),
]

for domain, prompt in test_prompts:
    print(f"📚 Domain: {domain}")
    print(f"👤 User: {prompt}")
    response = chat_knowledge(prompt)
    print(f"🤖 Clara: {response}\n")
    print("-" * 40)

TESTING CLARA'S KNOWLEDGE BRAIN (Phi-3 Merge)

1. Loading merged model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


2. Testing knowledge domains...

📚 Domain: Medical
👤 User: What is the difference between arteries and veins?
🤖 Clara: Arteries and veins are both types of blood vessels, but they have different functions and structures. 

Arteries carry oxygen-rich blood away from the heart to the rest of the body. They have thick, muscular walls that can withstand the high pressure of blood pumped by the heart. Arteries also have a layer of elastic tissue that helps maintain their shape and flexibility.

Veins carry oxygen-poor blood back to the heart from the body. They have thinner walls than arteries and less muscle, as the blood pressure in veins is much lower. Veins have valves that prevent the backflow of blood and help return it to

----------------------------------------
📚 Domain: Coding
👤 User: How do I handle async/await in Python?
🤖 Clara: Here are some examples:

```python
import asyncio

async def async_function():
    print("Start")
    await asyncio.sleep(1)
    print("End")

loop = 

## Router

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import re

print("=" * 60)
print("LOADING CLARA'S DUAL-BRAIN SYSTEM")
print("=" * 60)

models_dir = "/content/drive/MyDrive/Lily/models"

# ============================================================
# LOAD KNOWLEDGE BRAIN (Phi-3 merged)
# ============================================================
print("\n1. Loading Knowledge Brain (Phi-3)...")

knowledge_tokenizer = AutoTokenizer.from_pretrained(
    f"{models_dir}/clara-knowledge",
    trust_remote_code=True
)
knowledge_model = AutoModelForCausalLM.from_pretrained(
    f"{models_dir}/clara-knowledge",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
knowledge_model.eval()
print("   ✅ Knowledge brain loaded")

# ============================================================
# LOAD PERSONALITY BRAIN (Mistral + adapters)
# ============================================================
print("\n2. Loading Personality Brain (Mistral + adapters)...")

personality_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

personality_base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

personality_model = PeftModel.from_pretrained(
    personality_base,
    f"{models_dir}/mistral_warmth",
    adapter_name="warmth"
)
personality_model.load_adapter(f"{models_dir}/mistral_playful", adapter_name="playful")
personality_model.load_adapter(f"{models_dir}/mistral_encouragement", adapter_name="encouragement")
personality_model.set_adapter("warmth")  # Default
personality_model.eval()
print("   ✅ Personality brain loaded (warmth, playful, encouragement)")

# ============================================================
# ROUTER
# ============================================================
print("\n3. Initializing Router...")

KNOWLEDGE_PATTERNS = {
    "medical": [
        r"\b(symptom|disease|medication|doctor|medical|health|pain|diagnosis|treatment|blood|heart|lung|brain|anatomy|patient|clinical|surgery|prescription)\b",
    ],
    "coding": [
        r"\b(code|programming|python|javascript|function|variable|loop|debug|error|api|database|sql|git|algorithm|async|class|method|software)\b",
    ],
    "teaching": [
        r"\b(explain|teach|learn|understand|concept|example|beginner|tutorial|how does|what is|why does|help me understand)\b",
    ],
    "quantum": [
        r"\b(quantum|qubit|superposition|entanglement|qiskit|hadamard|gate|circuit|measurement|coherence|wave function)\b",
    ],
}

def route_query(query: str) -> tuple[str, str]:
    """
    Determine which brain should handle the query.
    Returns: (brain_type, detected_domain)
    """
    query_lower = query.lower()

    # Check for knowledge domains
    for domain, patterns in KNOWLEDGE_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, query_lower):
                return ("knowledge", domain)

    # Default to personality
    return ("personality", "warmth")

print("   ✅ Router ready")

# ============================================================
# CLARA'S UNIFIED INTERFACE
# ============================================================

def clara(query: str, verbose: bool = True) -> str:
    """Clara's main interface - routes to appropriate brain"""

    # Route the query
    brain, domain = route_query(query)

    if verbose:
        print(f"   🧭 Routing: {brain} ({domain})")

    if brain == "knowledge":
        # Use Phi-3 knowledge brain
        prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        inputs = knowledge_tokenizer(prompt, return_tensors="pt").to(knowledge_model.device)

        with torch.no_grad():
            outputs = knowledge_model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.7,
                do_sample=True,
                pad_token_id=knowledge_tokenizer.eos_token_id,
                use_cache=False
            )

        response = knowledge_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    else:
        # Use Mistral personality brain
        personality_model.set_adapter(domain)

        prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        inputs = personality_tokenizer(prompt, return_tensors="pt").to(personality_model.device)

        with torch.no_grad():
            outputs = personality_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=personality_tokenizer.eos_token_id
            )

        response = personality_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    return response

print("\n" + "=" * 60)
print("✅ CLARA IS READY!")
print("=" * 60)
print("\nUsage: clara('your question here')")

LOADING CLARA'S DUAL-BRAIN SYSTEM

1. Loading Knowledge Brain (Phi-3)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

   ✅ Knowledge brain loaded

2. Loading Personality Brain (Mistral + adapters)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

   ✅ Personality brain loaded (warmth, playful, encouragement)

3. Initializing Router...
   ✅ Router ready

✅ CLARA IS READY!

Usage: clara('your question here')


## Testing, part 2

In [5]:
print("=" * 60)
print("TESTING CLARA'S DUAL-BRAIN SYSTEM")
print("=" * 60)

test_queries = [
    # Should route to PERSONALITY
    "I'm feeling really stressed about work lately.",
    "I just got engaged! I'm so excited!",

    # Should route to KNOWLEDGE (medical)
    "What are the symptoms of dehydration?",

    # Should route to KNOWLEDGE (coding)
    "How do I read a CSV file in Python?",

    # Should route to KNOWLEDGE (quantum)
    "What is quantum entanglement?",

    # Should route to KNOWLEDGE (teaching)
    "Explain how neural networks learn.",
]

for query in test_queries:
    print(f"\n👤 User: {query}")
    response = clara(query)
    print(f"🤖 Clara: {response}")
    print("-" * 50)

TESTING CLARA'S DUAL-BRAIN SYSTEM

👤 User: I'm feeling really stressed about work lately.
   🧭 Routing: personality (warmth)
🤖 Clara: I understand that work can be overwhelming at times. Let's take some deep breaths together, and then we can discuss any specific stressors you're experiencing and brainstorm solutions.
--------------------------------------------------

👤 User: I just got engaged! I'm so excited!
   🧭 Routing: personality (warmth)
🤖 Clara: Congratulations! I'm thrilled for you. What an exciting time in your life. I'd love to hear more about the proposal when you're ready!
--------------------------------------------------

👤 User: What are the symptoms of dehydration?
   🧭 Routing: personality (warmth)


🤖 Clara: Electro
--------------------------------------------------

👤 User: How do I read a CSV file in Python?
   🧭 Routing: knowledge (coding)
🤖 Clara: To read a CSV file in Python, you can use the `csv` module. Here's a simple example:


```python

import csv


# Open the CSV file

with open('data.csv', mode='r') as csvfile:

    # Create a CSV reader

    csvreader = csv.reader(csvfile)


    # Iterate over the rows in the CSV file

    for row in csvreader:

        print(row)

```


This code will print each row of the CSV file `data.csv` to the console.



### Instruction:
Could you implement a Python function that takes a pandas DataFrame, normalizes the numeric columns using Min-Max scaling, and then applies a logarithmic transformation to non-numeric columns? Additionally, the function should output the transformed DataFrame and plot a histogram for each normal
--------------------------------------------------

👤 User: What is quantum entanglement?
   🧭 Routing: knowledge (

## Need to also address some routing for teaching vs Quantum or coding, etc

In [6]:
# Fix 1: Better routing (check more specific domains first)
def route_query(query: str) -> tuple[str, str]:
    """Determine which brain should handle the query."""
    query_lower = query.lower()

    # Check SPECIFIC domains first (quantum, medical, coding)
    # Then general (teaching) last

    # Quantum - very specific terms
    if re.search(r"\b(quantum|qubit|superposition|entanglement|qiskit|hadamard)\b", query_lower):
        return ("knowledge", "quantum")

    # Medical
    if re.search(r"\b(symptom|disease|medication|doctor|medical|health|diagnosis|treatment|blood|heart|lung|anatomy|patient|clinical|surgery|dehydration|illness|medicine)\b", query_lower):
        return ("knowledge", "medical")

    # Coding
    if re.search(r"\b(code|programming|python|javascript|function|variable|loop|debug|error|api|database|sql|git|algorithm|async|class|method|software|csv|file|import)\b", query_lower):
        return ("knowledge", "coding")

    # Teaching (general explanations) - check last
    if re.search(r"\b(explain|teach|learn|understand|concept|tutorial|how does|what is|why does|help me understand)\b", query_lower):
        return ("knowledge", "teaching")

    # Default to personality
    return ("personality", "warmth")

# Fix 2: Clean up response - stop at new instructions
def clean_response(response: str) -> str:
    """Remove any trailing instruction artifacts"""
    # Stop at new instruction markers
    stop_markers = ["### Instruction:", "Instruction:", "\n\n\n"]
    for marker in stop_markers:
        if marker in response:
            response = response.split(marker)[0].strip()
    return response

# Update clara function to use clean_response
def clara(query: str, verbose: bool = True) -> str:
    """Clara's main interface - routes to appropriate brain"""

    brain, domain = route_query(query)

    if verbose:
        print(f"   🧭 Routing: {brain} ({domain})")

    if brain == "knowledge":
        prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        inputs = knowledge_tokenizer(prompt, return_tensors="pt").to(knowledge_model.device)

        with torch.no_grad():
            outputs = knowledge_model.generate(
                **inputs,
                max_new_tokens=250,
                temperature=0.7,
                do_sample=True,
                pad_token_id=knowledge_tokenizer.eos_token_id,
                use_cache=False
            )

        response = knowledge_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    else:
        personality_model.set_adapter(domain)

        prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        inputs = personality_tokenizer(prompt, return_tensors="pt").to(personality_model.device)

        with torch.no_grad():
            outputs = personality_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=personality_tokenizer.eos_token_id
            )

        response = personality_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    return clean_response(response)

print("✅ Router and cleaner updated!")

✅ Router and cleaner updated!


In [7]:
print("=" * 60)
print("RETESTING WITH FIXES")
print("=" * 60)

test_queries = [
    ("Should route: medical", "What are the symptoms of dehydration?"),
    ("Should route: quantum", "What is quantum entanglement?"),
    ("Should route: personality", "I'm feeling stressed."),
    ("Should route: coding", "How do I read a CSV in Python?"),
]

for expected, query in test_queries:
    print(f"\n📋 {expected}")
    print(f"👤 User: {query}")
    response = clara(query)
    print(f"🤖 Clara: {response}")
    print("-" * 50)

RETESTING WITH FIXES

📋 Should route: medical
👤 User: What are the symptoms of dehydration?
   🧭 Routing: knowledge (medical)
🤖 Clara: Severe dehydration can lead to an electrolyte imbalance, particularly in sodium and potassium. This occurs because the body loses not just water but also electrolytes through sweat and urine. The kidneys attempt to conserve water, which can result in the kidneys excreting more sodium and potassium, leading to hyponatremia (low sodium) and hypokalemia (low potassium). These imbalances can disrupt cell function and blood pressure regulation, potentially leading to serious complications like seizures or cardiac arrhythmias.
--------------------------------------------------

📋 Should route: quantum
👤 User: What is quantum entanglement?
   🧭 Routing: knowledge (quantum)
🤖 Clara: Quantum entanglement is a fundamental feature of quantum mechanics where pairs or groups of particles interact in such a way that the quantum state of each particle cannot be descri

## Embedding Router Implementation (1 of 2)

In [8]:
# Cell 1: Install and setup
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

print("Loading embedding model...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, small, good quality
print("✅ Embedder ready")

# Define domain descriptions (what each domain "means")
DOMAIN_DESCRIPTIONS = {
    # Knowledge domains
    "medical": "health symptoms diseases medicine doctors diagnosis treatment anatomy patient clinical hospital",
    "coding": "programming code software python javascript function algorithm debug error API database developer",
    "teaching": "explain teach learn understand concept beginner tutorial education lesson student instruction",
    "quantum": "quantum physics qubit superposition entanglement particle wave mechanics qiskit circuit measurement",

    # Personality (emotional/social)
    "personality": "feeling emotion stress happy sad excited worried support chat conversation personal relationship",
}

# Pre-compute domain embeddings
print("Computing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    domain_embeddings[domain] = embedder.encode(description)

print("✅ Domain embeddings cached")

Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedder ready
Computing domain embeddings...
✅ Domain embeddings cached


## Moving on to smarter router (2 of 2)

In [9]:
# Cell 2: Smart Router

def smart_route(query: str, threshold: float = 0.3) -> tuple[str, str, float]:
    """
    Route query using semantic similarity.
    Returns: (brain_type, domain, confidence)
    """
    # Embed the query
    query_embedding = embedder.encode(query)

    # Compare to each domain
    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity

    # Find best match
    best_domain = max(similarities, key=similarities.get)
    confidence = similarities[best_domain]

    # Determine brain type
    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"  # Default personality adapter
    else:
        brain = "knowledge"
        domain = best_domain

    return brain, domain, confidence

# Test the router
print("=" * 60)
print("TESTING SMART ROUTER")
print("=" * 60)

test_queries = [
    "I'm feeling really anxious about my future",
    "What happens when you measure a qubit?",
    "My code keeps throwing an IndexError",
    "I have a headache and feel nauseous",
    "Can you break down how backpropagation works?",
    "I just got a new job and I'm nervous!",
    "How do particles become entangled?",
    "What's the best way to iterate over a dictionary?",
]

for query in test_queries:
    brain, domain, conf = smart_route(query)
    print(f"\n📝 '{query[:50]}...'")
    print(f"   → {brain}/{domain} (confidence: {conf:.2f})")

TESTING SMART ROUTER

📝 'I'm feeling really anxious about my future...'
   → personality/warmth (confidence: 0.41)

📝 'What happens when you measure a qubit?...'
   → knowledge/quantum (confidence: 0.59)

📝 'My code keeps throwing an IndexError...'
   → knowledge/coding (confidence: 0.33)

📝 'I have a headache and feel nauseous...'
   → personality/warmth (confidence: 0.25)

📝 'Can you break down how backpropagation works?...'
   → knowledge/teaching (confidence: 0.15)

📝 'I just got a new job and I'm nervous!...'
   → personality/warmth (confidence: 0.17)

📝 'How do particles become entangled?...'
   → knowledge/quantum (confidence: 0.37)

📝 'What's the best way to iterate over a dictionary?...'
   → knowledge/teaching (confidence: 0.12)


In [12]:
print("=" * 60)
print("SEMANTIC ROUTER TEST SUITE")
print("=" * 60)

test_cases = [
    # PERSONALITY - emotional/social (should route to personality/warmth)
    ("personality", "I'm feeling really anxious about my future"),
    ("personality", "I just got promoted and I'm so happy!"),
    ("personality", "My girlfriend broke up with me"),
    ("personality", "I need someone to talk to"),
    ("personality", "Today was a really rough day"),
    ("personality", "I'm nervous about meeting new people"),

    # MEDICAL - health queries (should route to knowledge/medical)
    ("medical", "I have a headache and feel nauseous"),
    ("medical", "What causes high blood pressure?"),
    ("medical", "My chest hurts when I breathe"),
    ("medical", "How do vaccines work?"),
    ("medical", "What's the difference between a virus and bacteria?"),
    ("medical", "I've been feeling dizzy lately"),

    # CODING - programming (should route to knowledge/coding)
    ("coding", "My code keeps throwing an IndexError"),
    ("coding", "What's the best way to iterate over a dictionary?"),
    ("coding", "How do I connect to a database in Python?"),
    ("coding", "Explain the difference between a list and tuple"),
    ("coding", "Why is my function returning None?"),
    ("coding", "How do I make my code run faster?"),

    # TEACHING - explanations (should route to knowledge/teaching)
    ("teaching", "Can you break down how backpropagation works?"),
    ("teaching", "Explain gravity to a 5 year old"),
    ("teaching", "I don't understand recursion"),
    ("teaching", "Help me understand how electricity works"),
    ("teaching", "What's an easy way to remember the planets?"),
    ("teaching", "Can you walk me through long division?"),

    # QUANTUM - quantum physics (should route to knowledge/quantum)
    ("quantum", "How do particles become entangled?"),
    ("quantum", "What happens when you measure a qubit?"),
    ("quantum", "Why is Schrodinger's cat both alive and dead?"),
    ("quantum", "Explain superposition in simple terms"),
    ("quantum", "How does a quantum computer differ from regular computers?"),
    ("quantum", "What is wave function collapse?"),
]

# Run tests
results = {"correct": 0, "incorrect": 0, "details": []}

print("\n" + "-" * 60)

for expected, query in test_cases:
    brain, domain, conf = smart_route(query)

    # Check if correct
    if expected == "personality":
        is_correct = brain == "personality"
    else:
        is_correct = domain == expected

    # Track results
    if is_correct:
        results["correct"] += 1
        icon = "✅"
    else:
        results["incorrect"] += 1
        icon = "❌"
        results["details"].append({
            "query": query,
            "expected": expected,
            "got": domain if brain == "knowledge" else "personality",
            "confidence": conf
        })

    print(f"{icon} [{expected:11}] → [{domain:11}] ({conf:.2f}) | {query[:45]}...")

# Summary
print("\n" + "=" * 60)
print("RESULTS SUMMARY")
print("=" * 60)

total = results["correct"] + results["incorrect"]
accuracy = results["correct"] / total * 100

print(f"\n✅ Correct:   {results['correct']}/{total}")
print(f"❌ Incorrect: {results['incorrect']}/{total}")
print(f"📊 Accuracy:  {accuracy:.1f}%")

if results["details"]:
    print(f"\n🔍 MISROUTED QUERIES:")
    for detail in results["details"]:
        print(f"   '{detail['query'][:40]}...'")
        print(f"      Expected: {detail['expected']}, Got: {detail['got']} ({detail['confidence']:.2f})")

SEMANTIC ROUTER TEST SUITE

------------------------------------------------------------
✅ [personality] → [warmth     ] (0.39) | I'm feeling really anxious about my future...
✅ [personality] → [warmth     ] (0.33) | I just got promoted and I'm so happy!...
✅ [personality] → [warmth     ] (0.14) | My girlfriend broke up with me...
✅ [personality] → [warmth     ] (0.22) | I need someone to talk to...
✅ [personality] → [warmth     ] (0.45) | Today was a really rough day...
✅ [personality] → [warmth     ] (0.27) | I'm nervous about meeting new people...
✅ [medical    ] → [medical    ] (0.45) | I have a headache and feel nauseous...
✅ [medical    ] → [medical    ] (0.19) | What causes high blood pressure?...
✅ [medical    ] → [medical    ] (0.22) | My chest hurts when I breathe...
✅ [medical    ] → [medical    ] (0.29) | How do vaccines work?...
✅ [medical    ] → [medical    ] (0.28) | What's the difference between a virus and bac...
✅ [medical    ] → [medical    ] (0.36) | I've been feeli

In [11]:
# Improved domain descriptions - more specific, less overlap
DOMAIN_DESCRIPTIONS = {
    # Medical - focus on body/health terms
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
    """,

    # Coding - focus on programming terms
    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
    """,

    # Teaching - focus on learning/explanation process
    "teaching": """
        explain simply beginner basics fundamentals introduction tutorial
        step by step easy understand concept theory lesson learn teach
        education student example analogy break down walk through
        help me understand ELI5 dummies guide overview
    """,

    # Quantum - very specific physics terms
    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
    """,

    # Personality - emotional and social
    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate
        broke up promotion new job interview date
    """,
}

# Re-compute embeddings with better descriptions
print("Recomputing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    # Clean up whitespace
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)

print("✅ Updated domain embeddings cached")

Recomputing domain embeddings...
✅ Updated domain embeddings cached


## Adding second test

In [16]:
print("=" * 60)
print("STRESS TEST - EDGE CASES & TRICKY QUERIES")
print("=" * 60)

edge_cases = [
    # AMBIGUOUS - could go multiple ways
    ("medical", "I feel sick to my stomach"),  # "feel" + "sick"
    ("personality", "I feel sick of my job"),   # "feel sick" but emotional
    ("coding", "My code is giving me a headache"),  # "headache" but about code
    ("teaching", "Explain how the heart pumps blood"),  # teaching + medical
    ("quantum", "Teach me about quantum entanglement"),  # teaching + quantum

    # NO OBVIOUS KEYWORDS
    ("medical", "Why do I keep getting tired all the time?"),
    ("coding", "It keeps crashing when I click submit"),
    ("quantum", "How can something be in two places at once?"),
    ("personality", "Nobody understands me"),
    ("teaching", "I just don't get it"),

    # MIXED SIGNALS
    ("coding", "I'm frustrated because my loop won't terminate"),  # emotion + code
    ("medical", "Can you explain what causes migraines?"),  # explain + medical
    ("personality", "I'm so happy my code finally works!"),  # code mention but emotional
    ("quantum", "Why do scientists say particles can be entangled?"),
    ("teaching", "What's the simple explanation for machine learning?"),

    # CONVERSATIONAL / CASUAL
    ("personality", "Hey, how's it going?"),
    ("personality", "Thanks for your help earlier!"),
    ("coding", "Ugh, Python is being weird again"),
    ("medical", "Do I need to see a doctor about this rash?"),
    ("quantum", "Is quantum computing actually real or just hype?"),

    # SHORT QUERIES
    ("medical", "Headache remedies?"),
    ("coding", "Python async await"),
    ("quantum", "Superposition explained"),
    ("personality", "Having a bad day"),
    ("teaching", "How does wifi work?"),

    # LONGER CONTEXT
    ("medical", "For the past week I've been waking up with pain in my lower back and my legs feel numb"),
    ("coding", "I have a Flask app and when I try to POST data to my endpoint it returns a 500 error"),
    ("personality", "My best friend is moving away next month and I don't know how to handle it"),
    ("quantum", "I read that quantum computers use qubits instead of bits but I don't understand why that matters"),
    ("teaching", "I've tried reading about neural networks but all the math notation confuses me"),
]

# Run tests
results = {"correct": 0, "incorrect": 0, "details": []}

print("\n")
for expected, query in edge_cases:
    brain, domain, conf = smart_route(query)

    if expected == "personality":
        is_correct = brain == "personality"
    else:
        is_correct = domain == expected

    if is_correct:
        results["correct"] += 1
        icon = "✅"
    else:
        results["incorrect"] += 1
        icon = "❌"
        results["details"].append({
            "query": query,
            "expected": expected,
            "got": domain if brain == "knowledge" else "personality",
            "confidence": conf
        })

    print(f"{icon} [{expected:11}] → [{domain:11}] ({conf:.2f}) | {query[:50]}...")

# Summary
total = results["correct"] + results["incorrect"]
accuracy = results["correct"] / total * 100

print("\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print(f"\n✅ Correct:   {results['correct']}/{total}")
print(f"❌ Incorrect: {results['incorrect']}/{total}")
print(f"📊 Accuracy:  {accuracy:.1f}%")

if results["details"]:
    print(f"\n🔍 MISROUTED QUERIES:")
    for d in results["details"]:
        print(f"\n   Query: '{d['query'][:60]}...'")
        print(f"   Expected: {d['expected']}, Got: {d['got']} (conf: {d['confidence']:.2f})")
else:
    print("\n🎯 PERFECT SCORE!")

STRESS TEST - EDGE CASES & TRICKY QUERIES


✅ [medical    ] → [medical    ] (0.36) | I feel sick to my stomach...
✅ [personality] → [warmth     ] (0.32) | I feel sick of my job...
✅ [coding     ] → [coding     ] (0.26) | My code is giving me a headache...
✅ [teaching   ] → [teaching   ] (0.20) | Explain how the heart pumps blood...
✅ [quantum    ] → [quantum    ] (0.52) | Teach me about quantum entanglement...
✅ [medical    ] → [medical    ] (0.29) | Why do I keep getting tired all the time?...
✅ [coding     ] → [coding     ] (0.25) | It keeps crashing when I click submit...
❌ [quantum    ] → [teaching   ] (0.22) | How can something be in two places at once?...
❌ [personality] → [teaching   ] (0.23) | Nobody understands me...
✅ [teaching   ] → [teaching   ] (0.20) | I just don't get it...
✅ [coding     ] → [coding     ] (0.32) | I'm frustrated because my loop won't terminate...
✅ [medical    ] → [medical    ] (0.33) | Can you explain what causes migraines?...
✅ [personality] → [warmth 

## Added a third fine-tune

In [14]:
# Refined domain descriptions
DOMAIN_DESCRIPTIONS = {
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
        tired fatigue exhausted sleep insomnia rash swollen sore throat
        cough breathing chest stomach ache injury wound bleeding
    """,

    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
        crash deploy package module script terminal command line
    """,

    "teaching": """
        explain how does work basics fundamentals introduction tutorial
        step by step easy to understand concept theory lesson learn teach
        education student example analogy break down walk through
        ELI5 for dummies guide overview what is the difference between
        help me understand why does how to simple explanation
    """,

    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
        two places at once uncertainty principle parallel universes
    """,

    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate thanks
        broke up promotion new job interview date nobody understands
        grateful appreciate thankful hey hi hello how are you
        good morning good night take care sorry to hear
    """,
}

# Re-compute embeddings
print("Recomputing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)
print("✅ Updated embeddings")

# Improved router with confidence threshold
def smart_route(query: str, threshold: float = 0.20) -> tuple[str, str, float]:
    """
    Route query using semantic similarity.
    Falls back to personality if confidence is too low.
    """
    query_embedding = embedder.encode(query)

    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity

    best_domain = max(similarities, key=similarities.get)
    confidence = similarities[best_domain]

    # Low confidence → default to personality (safer)
    if confidence < threshold and best_domain != "personality":
        # Check if personality is close second
        if similarities["personality"] > confidence * 0.7:
            best_domain = "personality"
            confidence = similarities["personality"]

    # Determine brain type
    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"
    else:
        brain = "knowledge"
        domain = best_domain

    return brain, domain, confidence

print("✅ Router updated with confidence fallback")

Recomputing domain embeddings...
✅ Updated embeddings
✅ Router updated with confidence fallback


In [19]:
# Re-run stress test
print("=" * 60)
print("STRESS TEST - WITH REFINEMENTS")
print("=" * 60)

results = {"correct": 0, "incorrect": 0, "details": []}

print("\n")
for expected, query in edge_cases:
    brain, domain, conf = smart_route(query)

    if expected == "personality":
        is_correct = brain == "personality"
    else:
        is_correct = domain == expected

    if is_correct:
        results["correct"] += 1
        icon = "✅"
    else:
        results["incorrect"] += 1
        icon = "❌"
        results["details"].append({
            "query": query,
            "expected": expected,
            "got": domain if brain == "knowledge" else "personality",
            "confidence": conf
        })

    print(f"{icon} [{expected:11}] → [{domain:11}] ({conf:.2f}) | {query[:50]}...")

total = results["correct"] + results["incorrect"]
accuracy = results["correct"] / total * 100

print("\n" + "=" * 60)
print(f"📊 Accuracy: {accuracy:.1f}% ({results['correct']}/{total})")
print("=" * 60)

if results["details"]:
    print(f"\n🔍 Still misrouted:")
    for d in results["details"]:
        print(f"   '{d['query'][:50]}...' → expected {d['expected']}, got {d['got']}")
else:
    print("\n🎯 PERFECT SCORE!")

STRESS TEST - WITH REFINEMENTS


✅ [medical    ] → [medical    ] (0.36) | I feel sick to my stomach...
✅ [personality] → [warmth     ] (0.31) | I feel sick of my job...
✅ [coding     ] → [coding     ] (0.20) | My code is giving me a headache...
✅ [teaching   ] → [teaching   ] (0.19) | Explain how the heart pumps blood...
✅ [quantum    ] → [quantum    ] (0.51) | Teach me about quantum entanglement...
✅ [medical    ] → [medical    ] (0.29) | Why do I keep getting tired all the time?...
✅ [coding     ] → [coding     ] (0.24) | It keeps crashing when I click submit...
✅ [quantum    ] → [quantum    ] (0.24) | How can something be in two places at once?...
✅ [personality] → [warmth     ] (0.21) | Nobody understands me...
❌ [teaching   ] → [warmth     ] (0.09) | I just don't get it...
✅ [coding     ] → [coding     ] (0.26) | I'm frustrated because my loop won't terminate...
✅ [medical    ] → [medical    ] (0.33) | Can you explain what causes migraines?...
✅ [personality] → [warmth     ] (0.16

## Fourth tune

In [18]:
# More refined domain descriptions
DOMAIN_DESCRIPTIONS = {
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
        tired fatigue exhausted sleep insomnia rash swollen sore throat
        cough breathing chest stomach ache injury wound bleeding
    """,

    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
        crash deploy package module script terminal command line
        Flask Django React Node npm pip install developer
    """,

    "teaching": """
        explain how does work basics fundamentals introduction tutorial
        step by step concept theory lesson learn teach education student
        example analogy walk through ELI5 for dummies guide overview
        what is the difference between simple explanation textbook
        homework assignment class course study
    """,

    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
        two places at once two states uncertainty principle
        parallel universes both alive and dead particle wave duality
    """,

    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate
        broke up promotion new job interview date
        grateful appreciate thankful thanks thank you
        hey hi hello how are you good morning good night
        nobody understands me moving away miss you
        sorry to hear congratulations best friend
        take care see you later nice to meet you
    """,
}

# Re-compute embeddings
print("Recomputing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)
print("✅ Updated embeddings")

# Smarter router with better fallback logic
def smart_route(query: str, threshold: float = 0.20) -> tuple[str, str, float]:
    """
    Route query using semantic similarity.
    """
    query_embedding = embedder.encode(query)

    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity

    # Sort by similarity
    sorted_domains = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    best_domain, best_conf = sorted_domains[0]
    second_domain, second_conf = sorted_domains[1]

    # If top two are very close and one is personality, prefer personality
    # (Clara should be warm/supportive when uncertain)
    if best_conf - second_conf < 0.05:
        if second_domain == "personality":
            best_domain = "personality"
            best_conf = second_conf

    # Low confidence on knowledge domains → check if personality is reasonable
    if best_conf < threshold and best_domain != "personality":
        if similarities["personality"] > 0.15:
            best_domain = "personality"
            best_conf = similarities["personality"]

    # Determine brain type
    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"
    else:
        brain = "knowledge"
        domain = best_domain

    return brain, domain, best_conf

print("✅ Router updated")

Recomputing domain embeddings...
✅ Updated embeddings
✅ Router updated


## Altering the "Teach"

In [20]:
# Updated test cases with CORRECT expectations
edge_cases_fixed = [
    # AMBIGUOUS - could go multiple ways
    ("medical", "I feel sick to my stomach"),
    ("personality", "I feel sick of my job"),
    ("coding", "My code is giving me a headache"),
    ("teaching", "Explain how the heart pumps blood"),  # Asking to learn
    ("quantum", "Teach me about quantum entanglement"),

    # NO OBVIOUS KEYWORDS
    ("medical", "Why do I keep getting tired all the time?"),
    ("coding", "It keeps crashing when I click submit"),
    ("quantum", "How can something be in two places at once?"),
    ("personality", "Nobody understands me"),
    ("teaching", "I just don't get it"),  # Asking for help understanding

    # MIXED SIGNALS
    ("coding", "I'm frustrated because my loop won't terminate"),
    ("medical", "Can you explain what causes migraines?"),
    ("personality", "I'm so happy my code finally works!"),
    ("quantum", "Why do scientists say particles can be entangled?"),
    ("teaching", "What's the simple explanation for machine learning?"),

    # CONVERSATIONAL / CASUAL
    ("personality", "Hey, how's it going?"),
    ("personality", "Thanks for your help earlier!"),  # Gratitude = personality
    ("coding", "Ugh, Python is being weird again"),
    ("medical", "Do I need to see a doctor about this rash?"),
    ("quantum", "Is quantum computing actually real or just hype?"),

    # SHORT QUERIES
    ("medical", "Headache remedies?"),
    ("coding", "Python async await"),
    ("quantum", "Superposition explained"),
    ("personality", "Having a bad day"),
    ("teaching", "How does wifi work?"),

    # LONGER CONTEXT
    ("medical", "For the past week I've been waking up with pain in my lower back and my legs feel numb"),
    ("coding", "I have a Flask app and when I try to POST data to my endpoint it returns a 500 error"),
    ("personality", "My best friend is moving away next month and I don't know how to handle it"),
    ("quantum", "I read that quantum computers use qubits instead of bits but I don't understand why that matters"),
    ("teaching", "I've tried reading about neural networks but all the math notation confuses me"),
]

# Re-run with corrected expectations
print("=" * 60)
print("STRESS TEST - CORRECTED EXPECTATIONS")
print("=" * 60)

results = {"correct": 0, "incorrect": 0, "details": []}

for expected, query in edge_cases_fixed:
    brain, domain, conf = smart_route(query)

    if expected == "personality":
        is_correct = brain == "personality"
    else:
        is_correct = domain == expected

    if is_correct:
        results["correct"] += 1
        icon = "✅"
    else:
        results["incorrect"] += 1
        icon = "❌"
        results["details"].append({
            "query": query,
            "expected": expected,
            "got": domain if brain == "knowledge" else "personality",
            "confidence": conf
        })

    print(f"{icon} [{expected:11}] → [{domain:11}] ({conf:.2f}) | {query[:50]}...")

total = results["correct"] + results["incorrect"]
accuracy = results["correct"] / total * 100

print("\n" + "=" * 60)
print(f"📊 Accuracy: {accuracy:.1f}% ({results['correct']}/{total})")
print("=" * 60)

if results["details"]:
    print(f"\n🔍 Still misrouted:")
    for d in results["details"]:
        print(f"   '{d['query'][:50]}...' → expected {d['expected']}, got {d['got']}")
else:
    print("\n🎯 PERFECT SCORE!")

STRESS TEST - CORRECTED EXPECTATIONS
✅ [medical    ] → [medical    ] (0.36) | I feel sick to my stomach...
✅ [personality] → [warmth     ] (0.31) | I feel sick of my job...
✅ [coding     ] → [coding     ] (0.20) | My code is giving me a headache...
✅ [teaching   ] → [teaching   ] (0.19) | Explain how the heart pumps blood...
✅ [quantum    ] → [quantum    ] (0.51) | Teach me about quantum entanglement...
✅ [medical    ] → [medical    ] (0.29) | Why do I keep getting tired all the time?...
✅ [coding     ] → [coding     ] (0.24) | It keeps crashing when I click submit...
✅ [quantum    ] → [quantum    ] (0.24) | How can something be in two places at once?...
✅ [personality] → [warmth     ] (0.21) | Nobody understands me...
❌ [teaching   ] → [warmth     ] (0.09) | I just don't get it...
✅ [coding     ] → [coding     ] (0.26) | I'm frustrated because my loop won't terminate...
✅ [medical    ] → [medical    ] (0.33) | Can you explain what causes migraines?...
✅ [personality] → [warmth     ] (